In [19]:
import os, sys, email
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4 as bs

import os
import itertools
from collections import Counter
from wordcloud import WordCloud
import re
import string

from tqdm import tqdm # for examining loops
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
import io
import pandas as pd


import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

from transformers import pipeline
import torch
torch.cuda.is_available()
from transformers import AutoTokenizer

import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
journal_df = pd.read_csv('data/journals.csv')
journal_df.head(1)

,authors,title,abstract,journal_name
0,Yu-Xiao Wang; Yue Xin; Jun-Yi Yin; Xiao-Jun Hu...,Revealing the architecture and solution proper...,Macrolepiota albuminosa (Berk.) Pegler is abun...,Food Chemistry


In [21]:
(journal_df.iloc[0]['abstract'])

'Macrolepiota albuminosa (Berk.) Pegler is abundant in active polysaccharides, but little is known about their structures and solution properties. In this study, water-extracted polysaccharides from M. albuminosa (MAWP) were purified into three fractions with structural heterogeneity, which was attributed to the diversity in molecular weight, monosaccharide composition and linkage patterns, further affecting their solution properties. Methylation and NMR analysis revealed MAWP-60p and MAWP-70 were a 3-O-methylated glucomannogalactan and a previously unreported glucomannogalactan, whereas MAWP-80 was elucidated as a branched galactoglucan. Besides, three fractions exhibited random coil conformation in aqueous solution, while MAWP-60p had the highest viscosity due to its highest molecular weight, mean square radius of gyration (Rg) and O-methyl group attached to the backbone. The molecular weight, monosaccharide composition and glycosidic linkages might be the major contributors to the f

## **Load Annotations**

In [22]:
import pandas as pd
import json
import os


with open('data/annotations/1.json', 'r') as f:
    data = json.load(f)

data

{'classes': ['MUSHROOM ',
  'CARBOHYDRATES ',
  'TECHNIQUE',
  'POLYSACCHARIDE',
  'PROPERTY',
  'ORG'],
 'annotations': [["'Macrolepiota albuminosa (Berk.) Pegler is abundant in active polysaccharides, but little is known about their structures and solution properties.\r",
   {'entities': [[0, 39, 'MUSHROOM '], [62, 77, 'POLYSACCHARIDE']]}],
  ['\r', {'entities': []}],
  [' In this study, water-extracted polysaccharides from M. albuminosa (MAWP) were purified into three fractions with structural heterogeneity, which was attributed to the diversity in molecular weight, monosaccharide composition and linkage patterns, further affecting their solution properties. \r',
   {'entities': [[53, 73, 'MUSHROOM '], [199, 213, 'CARBOHYDRATES ']]}],
  ['\r', {'entities': []}],
  ['Methylation and NMR analysis revealed MAWP-60p and MAWP-70 were a 3-O-methylated glucomannogalactan and a previously unreported glucomannogalactan, whereas MAWP-80 was elucidated as a branched galactoglucan. \r',
   {'en

In [23]:
train_data = data['annotations']
train_data = [tuple(i) for i in train_data]

entity_name = 'CHEMICAL'  # Replace with the desired entity name

for i in train_data:
    entities = i[1]['entities']
    if not entities:
        #i[1]['entities'] = (0, 0, entity_name)
        continue
    else:
        i[1]['entities'] = [tuple(entity) for entity in entities]

len(train_data)


11

In [24]:
directory = 'data/annotations/'  # Replace with the actual directory path
train_data = []

# Get a list of file names in the directory
file_names = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Iterate over each file
for file_name in file_names:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as f:
        data = json.load(f)
        annotations = data['annotations']
        train_data.extend(annotations)

train_data = [tuple(i) for i in train_data]

entity_name = 'CHEMICAL'  # Replace with the desired entity name

for i in train_data:
    entities = i[1]['entities']
    if not entities:
        continue
    else:
        i[1]['entities'] = [tuple(entity) for entity in entities]

len(train_data)

108

In [25]:
train_data = [
    (text, annot) for text, annot in train_data if annot['entities']
]
train_data

[("'Macrolepiota albuminosa (Berk.) Pegler is abundant in active polysaccharides, but little is known about their structures and solution properties.\r",
  {'entities': [(0, 39, 'MUSHROOM '), (62, 77, 'POLYSACCHARIDE')]}),
 (' In this study, water-extracted polysaccharides from M. albuminosa (MAWP) were purified into three fractions with structural heterogeneity, which was attributed to the diversity in molecular weight, monosaccharide composition and linkage patterns, further affecting their solution properties. \r',
  {'entities': [(53, 73, 'MUSHROOM '), (199, 213, 'CARBOHYDRATES ')]}),
 ('Methylation and NMR analysis revealed MAWP-60p and MAWP-70 were a 3-O-methylated glucomannogalactan and a previously unreported glucomannogalactan, whereas MAWP-80 was elucidated as a branched galactoglucan. \r',
  {'entities': [(0, 11, 'TECHNIQUE'),
    (16, 28, 'TECHNIQUE'),
    (38, 46, 'MUSHROOM '),
    (51, 58, 'MUSHROOM '),
    (66, 99, 'POLYSACCHARIDE'),
    (128, 146, 'POLYSACCHARIDE'),
   

## **Conversion of Data to .spacy format**

In [26]:
for text, annot in tqdm(train_data):
    print(text,annot)
    

100%|██████████| 58/58 [00:00<?, ?it/s]

 {'entities': [(0, 39, 'MUSHROOM '), (62, 77, 'POLYSACCHARIDE')]}ysaccharides, but little is known about their structures and solution properties.
 {'entities': [(53, 73, 'MUSHROOM '), (199, 213, 'CARBOHYDRATES ')]}MAWP) were purified into three fractions with structural heterogeneity, which was attributed to the diversity in molecular weight, monosaccharide composition and linkage patterns, further affecting their solution properties. 
 {'entities': [(0, 11, 'TECHNIQUE'), (16, 28, 'TECHNIQUE'), (38, 46, 'MUSHROOM '), (51, 58, 'MUSHROOM '), (66, 99, 'POLYSACCHARIDE'), (128, 146, 'POLYSACCHARIDE'), (156, 163, 'MUSHROOM '), (193, 206, 'POLYSACCHARIDE')]}
 {'entities': [(87, 95, 'MUSHROOM '), (112, 121, 'PROPERTY'), (141, 157, 'PROPERTY'), (159, 189, 'PROPERTY'), (191, 193, 'PROPERTY')]}ighest molecular weight, mean square radius of gyration (Rg) and O-methyl group attached to the backbone. 
 {'entities': [(4, 20, 'PROPERTY'), (22, 36, 'CARBOHYDRATES '), (53, 72, 'PROPERTY'), (125, 139, '

In [27]:
from spacy.tokens import DocBin

nlp = spacy.load("en_core_web_sm")
db = DocBin()  # create a DocBin object

for text, annot in tqdm(train_data):
    doc = nlp.make_doc(text)  # create doc object from text
    ents = []
    for start, end, label in annot['entities']:  # iterate over entities
        span = doc.char_span(start, end, label=label, alignment_mode='contract')
        if span is None:
            print('Skipping entity')
        else:
            ents.append(span)
    doc.ents = ents  # label the text with the entities
    db.add(doc)

db.to_disk('train.spacy')  # save the docbin object


100%|██████████| 58/58 [00:00<00:00, 257.93it/s]


In [28]:
nlp1 = spacy.load(r".\output\model-best") #load the best model
doc = nlp1(journal_df.iloc[3]['abstract']) # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [29]:
journal_df.iloc[120]['abstract']


'The nutrition and functionality of Chenopodium quinoa Willd are closely related to its components; however, these properties vary widely in different varieties of C. quinoa Willd from different origins. Using ultrahigh-performance liquid chromatography coupled with high-field quadrupole-orbitrap high-resolution mass spectrometry (UHPLC-QE HF HRMS) for metabolomics analysis, we performed nontargeted metabolomics analysis of C. quinoa Willd of different color varieties and origins and evaluated the texture characteristics after cooking. Both the variety and origin significantly affected the metabolic characteristics of C. quinoa Willd. The differences in the composition and texture properties were closely related to the lipid, amino acid, and polyphenol contents. The contents of kaempferol, catechol, myricetin 7-(6′-galloside), tyrosine, prostaglandin I2, and glutathione in white, red, and black quinoa significantly differed, and the variety had a greater effect on the lipid content tha

In [30]:
custom_ner = spacy.load(r".\output\model-best") #load the best model
doc = custom_ner(journal_df.iloc[125]['abstract']) # input sample text


In [31]:
def get_entities(df):

    # Apply text preprocessing and NER to the "abstract" column
    ner_results = []
    for abstract in df['abstract']:
        
        doc = custom_ner(abstract)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        ner_results.append(entities)

    # Add the cleaned abstract and NER results as new columns in the DataFrame
    #df['cleaned_abstract'] = df['abstract'].apply(preprocess_text)
    df['ner_results'] = ner_results

    return df

In [32]:
# Drop rows where the "abstract" column contains NaN values
journal_df = journal_df.dropna(subset=['abstract'])
journal_df[journal_df['abstract'].isna()] 

,authors,title,abstract,journal_name


In [33]:
journal_df = get_entities(journal_df)
journal_df.head()

,authors,title,abstract,journal_name,ner_results
0,Yu-Xiao Wang; Yue Xin; Jun-Yi Yin; Xiao-Jun Hu...,Revealing the architecture and solution proper...,Macrolepiota albuminosa (Berk.) Pegler is abun...,Food Chemistry,"[(Macrolepiota albuminosa (Berk.) Pegler, MUSH..."
1,Yu-Xiao Wang; Yue Xin; Xiao-Jun Huang; Jun-Yi ...,A branched galactoglucan with flexible chains ...,A homogeneous galactoglucan was purified from ...,Food Chemistry,"[(galactoglucan, POLYSACCHARIDE), (polysacchar..."
2,Yu-Xiao Wang; Ting Zhang; Jun-Yi Yin; Xiao-Jun...,Structural characterization and rheological pr...,A homogeneous beta-glucan (JHMP-70) was obtain...,Food Hydrocolloids,"[(beta-glucan, PROTEIN), (JHMP-70, MUSHROOM ),..."
3,Yu-Xin Gu; Tian-Ci Yan; Zi-Xuan Yue; Min-Hui L...,Dispersive micro-solid-phase extraction of aca...,A novel dispersive micro-solid-phase extractio...,Food Analytical Methods,"[(dispersive micro-solid-phase extraction, TEC..."
4,Yu-Xue Xu; Ze-Dong Jiang; Xi-Ping Du; Ming-Jin...,The identification of biotransformation pathwa...,The yeast Saccharomyces cerevisiae is effectiv...,Food Chemistry,"[(Saccharomyces cerevisiae, TECHNIQUE), (malod..."


In [34]:
journal_df.iloc[3]['abstract']

'A novel dispersive micro-solid-phase extraction (DMSPE) method is established for the determination of seven acaricides (clofentezine, benzomate, fenpyroximate, diafenthiuron, pyridaben, doramectin, and ivermectin) in fruit juice and functional food by ultra-high-performance liquid chromatography. In this method, these analytes were extracted from apple juice and Curcuma wenyujin using cucurbituril used as the adsorbent, water as the extraction solvent, and methanol as the elution solvent. The primary parameters affecting the extraction efficiency were optimized by experiments for concentration of adsorbent, oscillation time, desorption solvent, and pH value and by response surface methodology. Under the optimal conditions, all target analytes had a good linear relationship, with correlation coefficients ranging from 0.9900 to 0.9940, the limits of detection were 0.42-5.16 ng/mL, the limits of quantification were 1.39-17.18 ng/mL, and the spiked recoveries of seven target compounds in

In [51]:


ent_types = pd.read_csv('data/data_model/entity_types.csv')
ent_type_list = list(ent_types['Entity Type'])
ent_type_list.append('title')
ent_type_list

['ORG',
 'AGE GROUP',
 'PLANT SPECIES',
 'CONDITION',
 'DRINK',
 'LOC',
 'BACTERIUM',
 'CARBOHYDRATES ',
 'OBSERVATION',
 'PROPERTY',
 'SOLVENT',
 'CELL LINE',
 'MODE',
 'INSECT',
 'ORGAN',
 'TOXIN',
 'MEASUREMENT',
 'PROCESS',
 'FOOD',
 'PROTEIN',
 'STRUCTURE',
 'TECHNIQUE',
 'MATERIAL',
 'POLYSACCHARIDE',
 'ANIMAL MODEL',
 'MUSHROOM ',
 'APPLICATION',
 'CHEMICAL',
 'BIOLOGICAL PATHWAY',
 'MONOSACCHARIDE',
 'Title']

In [54]:
# Create an empty DataFrame to store the mapping
mapping_df = pd.DataFrame(columns=['Title', 'Entity', 'Entity_Type'])



entity_type_Df = pd.DataFrame(columns=ent_type_list)


# Iterate over each row in the DataFrame
for index, row in journal_df.iterrows():
    
    title = row['title']
    entity_pairs = row['ner_results']
    
    entity_list=[]
    entity_type_list=[]


    if len(entity_pairs) > 0:
        # Extract entities and entity types using list comprehension
        entities_list, entity_types_list = zip(*entity_pairs)
        entities = ','.join(entities_list)
        entity_types = ','.join(entity_types_list)

    

    # Append a new row to the mapping DataFrame
    mapping_df.loc[len(mapping_df)] = [title, entities, entity_types]
       

mapping_df.head(3)

,Title,Entity,Entity_Type
0,Revealing the architecture and solution proper...,"Macrolepiota albuminosa (Berk.) Pegler,polysac...","MUSHROOM ,POLYSACCHARIDE,MUSHROOM ,CARBOHYDRAT..."
1,A branched galactoglucan with flexible chains ...,"galactoglucan,polysaccharides,Macrolepiota alb...","POLYSACCHARIDE,POLYSACCHARIDE,MUSHROOM ,CHEMIC..."
2,Structural characterization and rheological pr...,"beta-glucan,JHMP-70,Hypsizygus marmoreus,ethan...","PROTEIN,MUSHROOM ,TOXIN,CHEMICAL,PROPERTY,APPL..."


In [ ]:
entity_type_Df

In [45]:
# mapping_df.to_csv('data/data_model/ner_outputs.csv',index=False)

## **Entity Types**

In [47]:
entity_types = set()

# Iterate through your dataset and collect unique entity types
for _, annotations in train_data:
    entities = annotations.get('entities')
    if entities:
        for entity in entities:
            entity_type = entity[2]
            entity_types.add(entity_type)

# Create a dataframe with entity type and entity type ID
df_entity_types = pd.DataFrame(data={'Entity Type': list(entity_types)})
df_entity_types['Entity Type ID'] = df_entity_types.index + 1

df_entity_types.to_csv('data/data_model/entity_types.csv',index=False)

## **Unique Entities**

In [ ]:
entity_types = set()

# Iterate through your dataset and collect unique entity types
for _, annotations in train_data:
    entities = annotations.get('entities')
    if entities:
        for entity in entities:
            entity_type = entity[2]
            entity_types.add(entity_type)

# Create a dataframe with entity type and entity type ID
df_entity_types = pd.DataFrame(data={'Entity Type': list(entity_types)})
df_entity_types['Entity Type ID'] = df_entity_types.index + 1

df_entity_types.to_csv('data/data_model/entity_types.csv',index=False)

## **Sentiment_analysis**

In [36]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer
torch.cuda.is_available()


True

In [37]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

# sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

sentiment_pipeline = pipeline("sentiment-analysis")


def label_sentiment(txt):

    return sentiment_pipeline(txt[:300])[0]['label'] 

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned

In [39]:
journal_df['sentiment'] = journal_df['abstract'].apply(label_sentiment)
journal_df.head()

,authors,title,abstract,journal_name,ner_results,sentiment
0,Yu-Xiao Wang; Yue Xin; Jun-Yi Yin; Xiao-Jun Hu...,Revealing the architecture and solution proper...,Macrolepiota albuminosa (Berk.) Pegler is abun...,Food Chemistry,"[(Macrolepiota albuminosa (Berk.) Pegler, MUSH...",NEGATIVE
1,Yu-Xiao Wang; Yue Xin; Xiao-Jun Huang; Jun-Yi ...,A branched galactoglucan with flexible chains ...,A homogeneous galactoglucan was purified from ...,Food Chemistry,"[(galactoglucan, POLYSACCHARIDE), (polysacchar...",NEGATIVE
2,Yu-Xiao Wang; Ting Zhang; Jun-Yi Yin; Xiao-Jun...,Structural characterization and rheological pr...,A homogeneous beta-glucan (JHMP-70) was obtain...,Food Hydrocolloids,"[(beta-glucan, PROTEIN), (JHMP-70, MUSHROOM ),...",NEGATIVE
3,Yu-Xin Gu; Tian-Ci Yan; Zi-Xuan Yue; Min-Hui L...,Dispersive micro-solid-phase extraction of aca...,A novel dispersive micro-solid-phase extractio...,Food Analytical Methods,"[(dispersive micro-solid-phase extraction, TEC...",POSITIVE
4,Yu-Xue Xu; Ze-Dong Jiang; Xi-Ping Du; Ming-Jin...,The identification of biotransformation pathwa...,The yeast Saccharomyces cerevisiae is effectiv...,Food Chemistry,"[(Saccharomyces cerevisiae, TECHNIQUE), (malod...",NEGATIVE


In [57]:
sentiment_df = journal_df[['title','sentiment']]
sentiment_df.rename(columns={'title':'Title'},inplace=True)
sentiment_df.head()

C:\Users\adity\AppData\Local\Temp\ipykernel_5952\3692506230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df.rename(columns={'title':'Title'},inplace=True)


,Title,sentiment
0,Revealing the architecture and solution proper...,NEGATIVE
1,A branched galactoglucan with flexible chains ...,NEGATIVE
2,Structural characterization and rheological pr...,NEGATIVE
3,Dispersive micro-solid-phase extraction of aca...,POSITIVE
4,The identification of biotransformation pathwa...,NEGATIVE


In [58]:
# sentiment_df.to_csv('data/data_model/sentiments.csv',index=False)

In [71]:
entity_dict = {}


for index, row in journal_df.iterrows():

        ner_result=row.ner_results

        for item in zip(ner_result):
                entity,e_type = item[0][0],item[0][1]
                if entity not in entity_dict:
                        entity_dict[entity] = e_type
                else:
                        continue

entity_data = []
entity_id = 1

for entity, entity_type in entity_dict.items():
    entity_data.append((entity_id, entity, entity_type))
    entity_id += 1

entity_df = pd.DataFrame(entity_data, columns=['Entity_ID', 'Entity', 'Entity_Type'])
entity_df.head()

,Entity_ID,Entity,Entity_Type
0,1,Macrolepiota albuminosa (Berk.) Pegler,MUSHROOM
1,2,polysaccharides,POLYSACCHARIDE
2,3,M. albuminosa (MAWP),MUSHROOM
3,4,monosaccharide,CARBOHYDRATES
4,5,Methylation,TECHNIQUE


In [72]:
entity_df.to_csv('data/data_model/entities.csv',index=False)